# Part 1

# Necessary Lists & Functions

In [108]:
import pandas as pd
import re

In [109]:
asean_list = ['VNM','LAO','THA','KHM','MYS','SGP','MMR','PHL','BRN','IDN']
south_asia_list = ['BGD','IND','PAK','NPL','LKA','BTN']
all_countries_list = ['VNM','LAO','THA','KHM','MYS','SGP','MMR','PHL','BRN','IDN','BGD','IND','PAK','NPL','LKA','BTN']

# Data Cleaning
def cleanUp(data):
    data = data[data['code'].isin(all_countries_list)]
    # Always decode all bytes in 'year' column if any exist
    if data['year'].apply(lambda x: isinstance(x, bytes)).any():
        data['year'] = data['year'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    first_year_value = data['year'].iloc[0]
    if isinstance(first_year_value, bytes):
        first_year_value = first_year_value.decode('utf-8')
    if re.match(r'^\d{4}-\d{2}-\d{2}$', str(first_year_value)):
        data['year'] = pd.to_datetime(data['year']).dt.year
    data = data.drop(data[data['year'] <= 2010].index)
    data = data.rename(columns={'code': 'country'})
    data = data.set_index(['country','year'])
    return data

def basic_stats(df):
    # Handle index vs columns for country/year
    if 'country' in df.index.names and 'year' in df.index.names:
        countries = df.index.get_level_values('country').unique().tolist()
        years = df.index.get_level_values('year')
    else:
        countries = df['country'].unique().tolist()
        years = df['year']
    
    # Calculate time range
    time_series = f"{years.min()} ~ {years.max()}"
    
    # Handle column dropping safely
    cols_to_drop = [col for col in ['country', 'year'] if col in df.columns]
    missing_data_summary = df.drop(cols_to_drop, axis=1).notna().sum()

    # get column count
    num_columns = df.shape[1]
    
    return {
        'countries (first 5)': countries[:5],
        'country_count': len(countries),
        'time_series': time_series,
        'variable_count': num_columns,
        'observations_bycolumn': missing_data_summary
    }


def merge_and_count(df_1, df_2, how = 'inner'):
    country_col = 'country'
    date_col = 'year'
    # Reset index if 'country' and 'year' are in the index
    if country_col in df_1.index.names and date_col in df_1.index.names:
        df_1 = df_1.reset_index()
    if country_col in df_2.index.names and date_col in df_2.index.names:
        df_2 = df_2.reset_index()
    rows_df_1, rows_df_2 = len(df_1), len(df_2)
    merged_df = pd.merge(df_1, df_2, how=how, on=[country_col, date_col])

    countries_df_1 = df_1[country_col].unique()
    countries_merged = merged_df[country_col].unique()

    dropped_countries = set(countries_df_1) - set(countries_merged)

    rows_merged = len(merged_df)
    countries_df_1_count, countries_merged_count = (
        len(countries_df_1),
        len(countries_merged)
    )

    print(f"Rows in df_1: {rows_df_1}, Countries in df_1: {countries_df_1_count}")
    print(f"Rows in df_2: {rows_df_2}, Countries in df_2: {countries_merged_count}")
    print(f"Rows in merged DataFrame: {rows_merged}, Countries in merged DataFrame: {countries_merged_count}")
    print(f"Dropped countries from df_1: {list(dropped_countries)}")

    return merged_df

def varialbe_summary(df):
    return pd.DataFrame({
        'type': df.dtypes,
        'non_missing': df.notna().sum(),
        'missing': df.isna().sum(),
        'nunique': df.nunique()
    })


# 1. Data sets

## 1.1 import all dataset

In [110]:
df_access = pd.read_csv("../data/processed/access_merged.csv")
df_controls = pd.read_csv("../data/processed/control_var.csv")
df_agriculture = pd.read_csv("../data/processed/agriculture_merged.csv")
df_staple = pd.read_csv("../data/processed/StapleFoodStability_adjusted.csv")
df_findex = pd.read_csv("../data/processed/cleaned_output_2011_2022.csv")
df_mobileTransaction = pd.read_csv('../Data/processed/mobile_transact.csv')


## 1.1 Agri (Agriculture Output)
- Source: World Bank FAO

In [111]:
df_agriculture = df_agriculture.rename(columns={'country':'code'})
df_agriculture_clean = cleanUp(df_agriculture)
df_agriculture_clean.head(10)



Agriculture, forestry, and fishing, value added (percent of GDP)  \
country year                                                                     
BGD     2024                                                NaN                  
        2023                                          11.003690                  
        2022                                          11.217601                  
        2021                                          11.632858                  
        2020                                          11.999687                  
        2019                                          11.975323                  
        2018                                          12.480077                  
        2017                                          12.957276                  
        2016                                          13.464792                  
        2015                                          14.782996                  

              Crop production index (2014-2016 = 100)  
country year                                           
BGD     2024                                      NaN  
        2023                                      NaN  
        2022                                   119.10  
        2021                                   116.46  
        2020                                   111.84  
        2019                                   109.89  
        2018                                   108.00  
        2017                                   108.47  
        2016                                   100.55  
        2015                                   100.69

In [112]:
basic_stats(df_agriculture_clean)

{'countries (first 5)': ['BGD', 'BTN', 'IND', 'LKA', 'NPL'],
 'country_count': 16,
 'time_series': '2011 ~ 2024',
 'variable_count': 2,
 'observations_bycolumn': Agriculture, forestry, and fishing, value added (percent of GDP)    207
 Crop production index (2014-2016 = 100)                             192
 dtype: int64}

- In this section, we examine agricultural output using data from the World Bank FAO. The dataset includes key indicators such as farm credit, ICT policy presence, agricultural production value, food and beverage processing value, and fertilizer usage. Our summary shows data coverage for 16 countries from 2000 to 2023. 

- Among these indicators, production and processing values have the most complete records, with 99 and 111 observations respectively. This suggests relatively strong data availability for measuring economic contributions of agricultural and food sectors, while access to farm credit and ICT policy information is more limited across countries and years.

## 1.2 Con (Control Variables)
- Source: WDI

In [113]:
df_controls = df_controls.drop(['country'], axis = 'columns')
df_control_clean = cleanUp(df_controls)
df_control_clean.head(3)

Fertility rate, total (births per woman)  \
country year                                             
BGD     2011                                     2.309   
        2012                                     2.263   
        2013                                     2.247   

              GDP per capita, PPP (constant 2021 international $)  \
country year                                                        
BGD     2011                                        4352.426434     
        2012                                        4592.048409     
        2013                                        4822.726214     

              Life expectancy at birth, total (years)  \
country year                                            
BGD     2011                                   68.499   
        2012                                   68.989   
        2013                                   69.487   

              Mortality rate, infant (per 1,000 live births)  \
country year                                                   
BGD     2011                                            35.4   
        2012                                            34.1   
        2013                                            32.7   

              Population density (people per sq. km of land area)  \
country year                                                        
BGD     2011                                        1179.926834     
        2012                                        1191.289091     
        2013                                        1202.520865     

              Population growth (annual %)  Population, total  \
country year                                                    
BGD     2011                      0.908526        153591076.0   
        2012                      0.958356        155070101.0   
        2013                      0.938408        156532141.0   

              Poverty headcount ratio at national poverty lines (% of population)  \
country year                                                                        
BGD     2011                                                NaN                     
        2012                                                NaN                     
        2013                                                NaN                     

              Rural population (% of total population)  \
country year                                             
BGD     2011                                    68.775   
        2012                                    68.007   
        2013                                    67.238   

              Urban population (% of total population)  
country year                                            
BGD     2011                                    31.225  
        2012                                    31.993  
        2013                                    32.762

In [114]:
basic_stats(df_control_clean)

{'countries (first 5)': ['BGD', 'BTN', 'BRN', 'KHM', 'IND'],
 'country_count': 16,
 'time_series': '2011 ~ 2023',
 'variable_count': 10,
 'observations_bycolumn': Fertility rate, total (births per woman)                               208
 GDP per capita, PPP (constant 2021 international $)                    207
 Life expectancy at birth, total (years)                                208
 Mortality rate, infant (per 1,000 live births)                         208
 Population density (people per sq. km of land area)                    192
 Population growth (annual %)                                           208
 Population, total                                                      208
 Poverty headcount ratio at national poverty lines (% of population)     52
 Rural population (% of total population)                               208
 Urban population (% of total population)                               208
 dtype: int64}

- In this section, we collect 10 control variables from WDI, covering economic development, population estimatesand composition, poverty prevalence, and vital statistics. Our summary shows data coverage for 16 countries from 1960 to 2023.

- Among these indicators, except poverty headcount ratio has many missing values, all other 9 variables have a considerable amount of data to serve as background variables. 

## 1.3 GFI (Global Financial Inclusion)
- Source: Findex

In [115]:
df_findex = df_findex.rename(columns={'country':'code'})
df_findex_clean = cleanUp(df_findex)
df_findex_clean.head(10)


Account (% age 15+)  \
country year                        
BGD     2011            31.740000   
        2012            31.490000   
        2013            31.240000   
        2014            30.990000   
        2015            37.343333   
        2016            43.696667   
        2017            50.050000   
        2018            50.740000   
        2019            51.430000   
        2020            52.120000   

              Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+)  \
country year                                                                                                                    
BGD     2011                                          31.740000                                                                 
        2012                                          31.490000                                                                 
        2013                                          31.240000                                                                 
        2014                                          30.990000                                                                 
        2015                                          37.343333                                                                 
        2016                                          43.696667                                                                 
        2017                                          50.050000                                                                 
        2018                                          50.740000                                                                 
        2019                                          51.430000                                                                 
        2020                                          52.120000                                                                 

              Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)  \
country year                                                                                                                            
BGD     2011                                          26.010000                                                                         
        2012                                          26.156667                                                                         
        2013                                          26.303333                                                                         
        2014                                          26.450000                                                                         
        2015                                          29.580000                                                                         
        2016                                          32.710000                                                                         
        2017                                          35.840000                                                                         
        2018                                          37.745000                                                                         
        2019                                          39.650000                                                                         
        2020                                          41.555000                                                                         

              Account ownership at a financial institution or with a mobile-money-service provider, male (% of population ages 15+)  \
country year                                                                                                                          
BGD     2011                                            37.2900                                          

In [116]:
basic_stats(df_findex_clean)

{'countries (first 5)': ['BGD', 'BRN', 'BTN', 'IDN', 'IND'],
 'country_count': 16,
 'time_series': '2011 ~ 2022',
 'variable_count': 108,
 'observations_bycolumn': Account (% age 15+)                                                                                                              180
 Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+)                  180
 Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)          180
 Account ownership at a financial institution or with a mobile-money-service provider, male (% of population ages 15+)            180
 Account ownership at a financial institution or with a mobile-money-service provider, older adults (% of population ages 25+)    180
                                                                                                                                 ... 
 Used a mobile phone or the inter

For the Global Financial Inclusion (GFI) dataset from Findex, we collect account ownership and digital financial service usage across 17 countries from 2011 to 2022. The dataset includes over 100 indicators detailing ownership by demographic breakdowns (e.g., gender, income, education), as well as behaviors such as using mobile phones or the internet to send money.

## 1.4 Staple (Staple Food Output)
- Source: FAO

In [117]:
df_staple_clean = cleanUp(df_staple)
df_staple_clean = df_staple_clean.rename(columns={'rolling_std': 'foodSupply_stability'})
df_staple.head(2)

,code,year,Food supply quantity (kg/capita/yr)_Cereals - Excluding Beer,Food supply quantity (kg/capita/yr)_Starchy Roots,rolling_std
0,AFG,2010,202.73,6.69,NaN
1,AFG,2011,197.29,5.72,NaN


In [118]:
basic_stats(df_staple_clean)

{'countries (first 5)': ['BGD', 'BTN', 'KHM', 'IND', 'IDN'],
 'country_count': 14,
 'time_series': '2011 ~ 2022',
 'variable_count': 3,
 'observations_bycolumn': Food supply quantity (kg/capita/yr)_Cereals - Excluding Beer    160
 Food supply quantity (kg/capita/yr)_Starchy Roots               160
 foodSupply_stability                                            160
 dtype: int64}

For the staple food dataset, we collect 18 countries from 2010-2022 over three indicators, food supply quantity for cereals, food supply quantity for starch, standard deviations. 

## 1.5 Access (to digital finance)
- Source: WDI

In [119]:
df_access = df_access.rename(columns={'country':'code'})
df_access_clean = cleanUp(df_access)
df_access_clean.head(2)

Rural Access to Electricity(Percent of Population)  \
country year                                                       
BGD     2024                                                NaN    
        2023                                               99.6    

              Mobile Cellular Subscriptions (per 100 people)  \
country year                                                   
BGD     2024                                             NaN   
        2023                                             NaN   

              Fixed Broadband Subsciptions (per 100 people)  
country year                                                 
BGD     2024                                            NaN  
        2023                                        7.51447

In [120]:
basic_stats(df_access_clean)

{'countries (first 5)': ['BGD', 'BTN', 'IND', 'LKA', 'NPL'],
 'country_count': 16,
 'time_series': '2011 ~ 2024',
 'variable_count': 3,
 'observations_bycolumn': Rural Access to Electricity(Percent of Population)    204
 Mobile Cellular Subscriptions (per 100 people)        190
 Fixed Broadband Subsciptions (per 100 people)         201
 dtype: int64}

For the access to digital finance from WDI, we collect 16 countries from 1960-2024, covering variables like rural access to electricity, mobile subscriptions, and fixed broadband subscriptions. Among which, mobile sucscription has the largest amount of available data for us to use.

## 1.6 Mobile (Mobile Money Transactions)
- Source: Financial Access Survey, IMF

In [121]:
df_mobileTransaction = df_mobileTransaction[['code', 'year', 'mobile_money_transactions']]
df_mobileTransaction_clean = cleanUp(df_mobileTransaction)
df_mobileTransaction_clean.head(2)


mobile_money_transactions
country year                           
IDN     2016               7.063689e+06
        2017               1.237547e+07

In [122]:
basic_stats(df_mobileTransaction_clean)

{'countries (first 5)': ['IDN', 'MMR', 'BGD', 'PAK', 'PHL'],
 'country_count': 15,
 'time_series': '2016 ~ 2023',
 'variable_count': 1,
 'observations_bycolumn': mobile_money_transactions    132
 dtype: int64}

For the mobile money transactions dataset, we collected data from 19 countries in South and Southeast Asia spanning the years 2016 to 2023. The indicator used is the total number of mobile money transactions per year, resulting in 152 valid observations. This subset helps us capture recent trends in digital financial inclusion across the region.

## 1.7 Fertilizer use (Nitrogen application per farmland)
- Source: Global data on fertilizer use by crop and by country, DRYAD(https://datadryad.org/dataset/doi:10.5061/dryad.2rbnzs7qh)
- not applied due to data limitation

# 2. Merge Datasets

## 2.1 Merge control and agri

In [123]:
df_merge_1 = merge_and_count(
    df_1 = df_control_clean,
    df_2 = df_agriculture_clean,
    how = 'outer'
)
# df_merge_1.head(2)
basic_stats(df_merge_1)

Rows in df_1: 208, Countries in df_1: 16
Rows in df_2: 224, Countries in df_2: 16
Rows in merged DataFrame: 224, Countries in merged DataFrame: 16
Dropped countries from df_1: []


{'countries (first 5)': ['BGD', 'BRN', 'BTN', 'IDN', 'IND'],
 'country_count': 16,
 'time_series': '2011 ~ 2024',
 'variable_count': 14,
 'observations_bycolumn': Fertility rate, total (births per woman)                               208
 GDP per capita, PPP (constant 2021 international $)                    207
 Life expectancy at birth, total (years)                                208
 Mortality rate, infant (per 1,000 live births)                         208
 Population density (people per sq. km of land area)                    192
 Population growth (annual %)                                           208
 Population, total                                                      208
 Poverty headcount ratio at national poverty lines (% of population)     52
 Rural population (% of total population)                               208
 Urban population (% of total population)                               208
 Agriculture, forestry, and fishing, value added (percent of GDP)       207
 C

## 2.2 Merge df_merge1 and GFI

In [124]:
df_merge_2 = merge_and_count(
    df_1 = df_merge_1,
    df_2 = df_findex_clean,
    how = 'outer'
)
# df_merge_2.head(2)
basic_stats(df_merge_2)

Rows in df_1: 224, Countries in df_1: 16
Rows in df_2: 192, Countries in df_2: 16
Rows in merged DataFrame: 224, Countries in merged DataFrame: 16
Dropped countries from df_1: []


{'countries (first 5)': ['BGD', 'BRN', 'BTN', 'IDN', 'IND'],
 'country_count': 16,
 'time_series': '2011 ~ 2024',
 'variable_count': 122,
 'observations_bycolumn': Fertility rate, total (births per woman)                                 208
 GDP per capita, PPP (constant 2021 international $)                      207
 Life expectancy at birth, total (years)                                  208
 Mortality rate, infant (per 1,000 live births)                           208
 Population density (people per sq. km of land area)                      192
                                                                         ... 
 Used a mobile phone or the internet to send money (% age 15+)            168
 Used a mobile phone or the internet to send money, female (% age 15+)    168
 Used a mobile phone or the internet to send money, male (% age 15+)      168
 Used a mobile phone or the internet to send money, rural (% age 15+)      96
 Used a mobile phone or the internet to send money, urban

## 2.3 Merge df_merge2 and staple

In [125]:
df_merge_3 = merge_and_count(
    df_1 = df_merge_2,
    df_2 = df_staple_clean,
    how = 'outer'
)
basic_stats(df_merge_3)

Rows in df_1: 224, Countries in df_1: 16
Rows in df_2: 160, Countries in df_2: 16
Rows in merged DataFrame: 224, Countries in merged DataFrame: 16
Dropped countries from df_1: []


{'countries (first 5)': ['BGD', 'BRN', 'BTN', 'IDN', 'IND'],
 'country_count': 16,
 'time_series': '2011 ~ 2024',
 'variable_count': 125,
 'observations_bycolumn': Fertility rate, total (births per woman)                                208
 GDP per capita, PPP (constant 2021 international $)                     207
 Life expectancy at birth, total (years)                                 208
 Mortality rate, infant (per 1,000 live births)                          208
 Population density (people per sq. km of land area)                     192
                                                                        ... 
 Used a mobile phone or the internet to send money, rural (% age 15+)     96
 Used a mobile phone or the internet to send money, urban (% age 15+)    108
 Food supply quantity (kg/capita/yr)_Cereals - Excluding Beer            160
 Food supply quantity (kg/capita/yr)_Starchy Roots                       160
 foodSupply_stability                                              

## 2.4 Merge df_merge3 and access

In [126]:
df_merge_4 = merge_and_count(
    df_1 = df_merge_3,
    df_2 = df_access_clean,
    how = 'outer'
)
basic_stats(df_merge_4)


Rows in df_1: 224, Countries in df_1: 16
Rows in df_2: 224, Countries in df_2: 16
Rows in merged DataFrame: 224, Countries in merged DataFrame: 16
Dropped countries from df_1: []


{'countries (first 5)': ['BGD', 'BRN', 'BTN', 'IDN', 'IND'],
 'country_count': 16,
 'time_series': '2011 ~ 2024',
 'variable_count': 128,
 'observations_bycolumn': Fertility rate, total (births per woman)               208
 GDP per capita, PPP (constant 2021 international $)    207
 Life expectancy at birth, total (years)                208
 Mortality rate, infant (per 1,000 live births)         208
 Population density (people per sq. km of land area)    192
                                                       ... 
 Food supply quantity (kg/capita/yr)_Starchy Roots      160
 foodSupply_stability                                   160
 Rural Access to Electricity(Percent of Population)     204
 Mobile Cellular Subscriptions (per 100 people)         190
 Fixed Broadband Subsciptions (per 100 people)          201
 Length: 126, dtype: int64}

## 2.5 Merge df_merge4 and mobile

In [127]:
df_merge_5 = merge_and_count(
    df_1 = df_merge_4,
    df_2 = df_mobileTransaction_clean,
    how = 'outer'
)
basic_stats(df_merge_5)


Rows in df_1: 224, Countries in df_1: 16
Rows in df_2: 232, Countries in df_2: 16
Rows in merged DataFrame: 336, Countries in merged DataFrame: 16
Dropped countries from df_1: []


{'countries (first 5)': ['BGD', 'BRN', 'BTN', 'IDN', 'IND'],
 'country_count': 16,
 'time_series': '2011 ~ 2024',
 'variable_count': 129,
 'observations_bycolumn': Fertility rate, total (births per woman)               320
 GDP per capita, PPP (constant 2021 international $)    318
 Life expectancy at birth, total (years)                320
 Mortality rate, infant (per 1,000 live births)         320
 Population density (people per sq. km of land area)    290
                                                       ... 
 foodSupply_stability                                   241
 Rural Access to Electricity(Percent of Population)     312
 Mobile Cellular Subscriptions (per 100 people)         286
 Fixed Broadband Subsciptions (per 100 people)          309
 mobile_money_transactions                              132
 Length: 127, dtype: int64}

## 2.6 save merged file

In [128]:
df_merge_5.to_csv('../Data/processed/merged_final.csv', index=False)